In [1]:
# import libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings

warnings.filterwarnings("ignore")
colors = sns.color_palette("Accent")

# Exploration Findings

1. **Dataset Overview**
   - The data has `541909 rows × 8 columns`

2. **Missing Values**
   - 24.93% of `CustomerID` is missing, we will just have to remove them, as there is no relation to identify why this might happend
   - 0.27% of `Description` is missing, but we doesn't need it any way, so we will just remove the column for now
   
3. **Feature Distributions**

4. **Relationships**
    
5. **Outliers**

6. **Time Trends**

7. **Preprocessing notes**
   - Change `InvoiceDate` to datatime
   - Change `CustomerID` to object
   - Drop null values in `CustomerID`
   - Drop duplicated values

8. **Feature engineering notes**
   - Remove `Description` column
   - Add `TotalSpent` column, ((UnitPrice * Quantity) for each customer)
   - Add `TotaclCancelation` (The number of cancelations for each customer)
   - Add `Frequency` column (The number of invoices for each customer)
   - Add `ItemsBought` column (The number of items bought for each customer)
   - Add `Recency` column (The time since the last purchase for each customer)
   - Add `LifeTime` column (The time differnce between first and last purchase)
   - Add `AverageValue` column (Total revenue divided by the number of transactions)
   - Add `ProductDiversity` column (Count of unique products purchased)


9. **Labeling notes**
   - We need to encode the type of gender to one hot enconding


## Loading data
   - Change `InvoiceDate` to datatime
   - Change `CustomerID` to object

In [2]:
# Load data
df = pd.read_csv("data.csv", encoding="ISO-8859-1")

In [3]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
# Change data types
df["CustomerID"] = df["CustomerID"].dropna().astype("int").astype("str")
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  object        
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 33.1+ MB


In [7]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


In [8]:
# null values
number_of_null_values = df.isnull().sum()
percentage = round((number_of_null_values / len(df)) * 100, 2)

null_frame = pd.DataFrame(
    {
        "number_of_null": number_of_null_values.values,
        "percentage": percentage,
    }
)

null_frame

,number_of_null,percentage
InvoiceNo,0,0.00
StockCode,0,0.00
Description,1454,0.27
Quantity,0,0.00
InvoiceDate,0,0.00
UnitPrice,0,0.00
CustomerID,135080,24.93
Country,0,0.00


There are a lot of null values in cusomter Id, we will invistigate them then we will decide what to do

## Invistigate null values
I want to see if there is any patterns with null values in customerID

In [9]:
null_customer_df = df[df["CustomerID"].isnull()]
null_customer_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,2011-12-09 10:26:00,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,2011-12-09 10:26:00,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,2011-12-09 10:26:00,10.79,NaN,United Kingdom


In [10]:
"""
invoices_with_more_than_customer = []

# This loop would go through all null values in the data, to see if the invoice has another id associated with it
# To test if the null value is beacuse that the customer id was written with the same invoice once
for index in null_customer_df.index:
    print(index)
    if (
        len(df[df["InvoiceNo"] == df["InvoiceNo"].iloc[index]]["CustomerID"].unique())
        > 1
    ):
        invoices_with_more_than_customer.append(df["InvoiceNo"].iloc[index])
        """


'\ninvoices_with_more_than_customer = []\n\n# This loop would go through all null values in the data, to see if the invoice has another id associated with it\n# To test if the null value is beacuse that the customer id was written with the same invoice once\nfor index in null_customer_df.index:\n    print(index)\n    if (\n        len(df[df["InvoiceNo"] == df["InvoiceNo"].iloc[index]]["CustomerID"].unique())\n        > 1\n    ):\n        invoices_with_more_than_customer.append(df["InvoiceNo"].iloc[index])\n        '

We have no relation

In [11]:
# We will drop the values of cusomerID
df = df.dropna(subset=["CustomerID"])

## Duplicates
- We Have to drop duplicates

In [12]:
# Mark all duplicates as True, including the first occurrence
duplicates = df.duplicated(keep=False)

# Filter the DataFrame to show only the duplicated rows
duplicated_rows = df[duplicates]

# Sort the DataFrame to group duplicates together
duplicated_rows = duplicated_rows.sort_values(by=df.columns.tolist())

# Display the duplicated rows
duplicated_rows

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
494,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908,United Kingdom
517,536409,21866,UNION JACK FLAG LUGGAGE TAG,1,2010-12-01 11:45:00,1.25,17908,United Kingdom
485,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908,United Kingdom
539,536409,22111,SCOTTIE DOG HOT WATER BOTTLE,1,2010-12-01 11:45:00,4.95,17908,United Kingdom
489,536409,22866,HAND WARMER SCOTTY DOG DESIGN,1,2010-12-01 11:45:00,2.10,17908,United Kingdom
...,...,...,...,...,...,...,...,...
440149,C574510,22360,GLASS JAR ENGLISH CONFECTIONERY,-1,2011-11-04 13:25:00,2.95,15110,United Kingdom
461407,C575940,23309,SET OF 60 I LOVE LONDON CAKE CASES,-24,2011-11-13 11:38:00,0.55,17838,United Kingdom
461408,C575940,23309,SET OF 60 I LOVE LONDON CAKE CASES,-24,2011-11-13 11:38:00,0.55,17838,United Kingdom
529980,C580764,22667,RECIPE BOX RETROSPOT,-12,2011-12-06 10:38:00,2.95,14562,United Kingdom


In [13]:
df = df.drop_duplicates()

In [14]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


## Further analysis

In [50]:
df.describe()

,Quantity,InvoiceDate,UnitPrice
count,401604.000000,401604,401604.000000
mean,12.183273,2011-07-10 12:08:23.848567552,3.474064
min,-80995.000000,2010-12-01 08:26:00,0.000000
25%,2.000000,2011-04-06 15:02:00,1.250000
50%,5.000000,2011-07-29 15:40:00,1.950000
75%,12.000000,2011-10-20 11:58:30,3.750000
max,80995.000000,2011-12-09 12:50:00,38970.000000
std,250.283037,NaN,69.764035


We see that there are negative values in the quantity, maybe this is happend as the customer brought back something he bought, but we have to make sure

In [54]:
negative_df = df[df["Quantity"] < 0]
negative_df[negative_df["CustomerID"] == "15311"]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
18196,C537805,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-08 13:18:00,4.65,15311,United Kingdom
18197,C537805,40001,WHITE BAMBOO RIBS LAMPSHADE,-2,2010-12-08 13:18:00,0.64,15311,United Kingdom
18198,C537805,85047,WHITE BEADED GARLAND STRING 20LIGHT,-1,2010-12-08 13:18:00,5.55,15311,United Kingdom
18199,C537805,22719,GUMBALL MONOCHROME COAT RACK,-3,2010-12-08 13:18:00,1.06,15311,United Kingdom
...,...,...,...,...,...,...,...,...
468960,C576561,21929,JUMBO BAG PINK VINTAGE PAISLEY,-1,2011-11-15 13:22:00,1.79,15311,United Kingdom
468961,C576561,22411,JUMBO SHOPPER VINTAGE RED PAISLEY,-2,2011-11-15 13:22:00,1.79,15311,United Kingdom
510077,C579415,23169,CLASSIC GLASS COOKIE JAR,-4,2011-11-29 12:52:00,3.75,15311,United Kingdom
530911,C580886,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-06 12:25:00,10.95,15311,United Kingdom


In [57]:
# The sum of the prices for one customer should be more than 0
# So we will create a for loop to go through all the customers and check if the sum of the prices is more than 0

negative_customer = negative_df["CustomerID"].unique()
sum_list = []

for customer in negative_customer:
    sum = df[df["CustomerID"] == customer]["UnitPrice"].sum()
    sum_list.append(sum)

sumed_df = pd.DataFrame({"CustomerID": negative_customer, "Sum": sum_list})

In [59]:
sumed_df[sumed_df["Sum"] < 0]

,CustomerID,Sum


We made sure that the sum of all unitprices is postive

## Feature engineering
- Remove `Description` column
- Add `TotalPrice` column, ((UnitPrice * Quantity) for each customer)
- Add `TotaclCancelation` (The number of cancelations for each customer)
- Add `TotalOrders` column (The number of orders for each customer, the cancelation is not included)
- Add `ItemsBought` column (The number of items bought for each customer)
- Add `Recency` column (The time since the last purchase for each customer, in days)
- Add `LifeTime` column (The time differnce between first and last purchase, in number of days)
- Add `AverageValue` column (Total revenue divided by the number of transactions)
- Add `ProductDiversity` column (Count of unique products purchased)
- Add `CommonDay` column (Mode day for each customer)
- Add `CommonHour` column (Mode hour for each customer)
- Add Country to the customer profile

In [19]:
print(f"The number of unique customers is {df['CustomerID'].nunique()}")
print(f"The number of unique products is {df['StockCode'].nunique()}")

The number of unique customers is 4372
The number of unique products is 3684


In [20]:
# Remove the Description column
df = df.drop("Description", axis=1)

In [35]:
# Add TotalPrice column
TotalPrice = []
for index in range(len(df)):
    TotalPrice.append(df["Quantity"].iloc[index] * df["UnitPrice"].iloc[index])

df["TotalPrice"] = TotalPrice
df

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
...,...,...,...,...,...,...,...,...
541904,581587,22613,12,2011-12-09 12:50:00,0.85,12680,France,10.20
541905,581587,22899,6,2011-12-09 12:50:00,2.10,12680,France,12.60
541906,581587,23254,4,2011-12-09 12:50:00,4.15,12680,France,16.60
541907,581587,23255,4,2011-12-09 12:50:00,4.15,12680,France,16.60


In [59]:
# Add column to identify if the invoice is canceled or not
df["Canceled"] = df["InvoiceNo"].apply(lambda x: 1 if "C" in x else 0)
df

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Canceled
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,0
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,0
2,536365,84406B,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,0
3,536365,84029G,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,0
4,536365,84029E,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,0
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,12,2011-12-09 12:50:00,0.85,12680,France,10.20,0
541905,581587,22899,6,2011-12-09 12:50:00,2.10,12680,France,12.60,0
541906,581587,23254,4,2011-12-09 12:50:00,4.15,12680,France,16.60,0
541907,581587,23255,4,2011-12-09 12:50:00,4.15,12680,France,16.60,0


In [63]:
# Add day, and hour columns
df["Day"] = df["InvoiceDate"].dt.day_name()
df["Hour"] = df["InvoiceDate"].dt.hour.astype(
    "object"
)  # Add the hour and make it object
df

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Canceled,Day,Hour
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,0,Wednesday,8
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,0,Wednesday,8
2,536365,84406B,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,0,Wednesday,8
3,536365,84029G,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,0,Wednesday,8
4,536365,84029E,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,0,Wednesday,8
...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,12,2011-12-09 12:50:00,0.85,12680,France,10.20,0,Friday,12
541905,581587,22899,6,2011-12-09 12:50:00,2.10,12680,France,12.60,0,Friday,12
541906,581587,23254,4,2011-12-09 12:50:00,4.15,12680,France,16.60,0,Friday,12
541907,581587,23255,4,2011-12-09 12:50:00,4.15,12680,France,16.60,0,Friday,12


In [87]:
# Group columns by the CustomerID
grouped_df = df.groupby("CustomerID").agg(
    {
        "InvoiceNo": "nunique",  # Number of unique invoices
        "TotalPrice": ["sum", "mean"],  # Total and average Spent
        "Quantity": "sum",  # Total Quantity
        "Canceled": "sum",  # Number of canceled invoices
        "Day": lambda x: x.mode().iloc[0],  # Most frequent day
        "Hour": lambda x: x.mode().iloc[0],  # Most frequent hour
        "StockCode": "nunique",  # Number of unique products
        "InvoiceDate": ["min", "max"],  # First and last purchase
        "Country": lambda x: x.mode().iloc[0],  # Most frequent country
    }
)
grouped_df

InvoiceNo TotalPrice            Quantity Canceled        Day  \
             nunique        sum       mean      sum      sum   <lambda>   
CustomerID                                                                
12346              2       0.00   0.000000        0        1    Tuesday   
12347              7    4310.00  23.681319     2458        0    Tuesday   
12348              4    1797.24  57.975484     2341        0   Thursday   
12349              1    1757.55  24.076027      631        0     Monday   
12350              1     334.40  19.670588      197        0  Wednesday   
...              ...        ...        ...      ...      ...        ...   
18280              1     180.60  18.060000       45        0     Monday   
18281              1      80.82  11.545714       54        0     Sunday   
18282              3     176.60  13.584615       98        1     Friday   
18283             16    2045.53   2.837074     1357        0   Thursday   
18287              3    1837.28  26.246857     1586        0  Wednesday   

               Hour StockCode         InvoiceDate                      \
           <lambda>   nunique                 min                 max   
CustomerID                                                              
12346            10         1 2011-01-18 10:01:00 2011-01-18 10:17:00   
12347            14       103 2010-12-07 14:57:00 2011-12-07 15:52:00   
12348            19        22 2010-12-16 19:09:00 2011-09-25 13:13:00   
12349             9        73 2011-11-21 09:51:00 2011-11-21 09:51:00   
12350            16        17 2011-02-02 16:01:00 2011-02-02 16:01:00   
...             ...       ...                 ...                 ...   
18280             9        10 2011-03-07 09:52:00 2011-03-07 09:52:00   
18281            10         7 2011-06-12 10:53:00 2011-06-12 10:53:00   
18282            13        12 2011-08-05 13:35:00 2011-12-02 11:43:00   
18283            14       263 2011-01-06 14:14:00 2011-12-06 12:02:00   
18287            10        59 2011-05-22 10:39:00 2011-10-28 09:29:00   

                   Country  
                  <lambda>  
CustomerID                  
12346       United Kingdom  
12347              Iceland  
12348              Finland  
12349                Italy  
12350               Norway  
...                    ...  
18280       United Kingdom  
18281       United Kingdom  
18282       United Kingdom  
18283       United Kingdom  
18287       United Kingdom  

[4372 rows x 11 columns]

In [88]:
# Chnage column names
grouped_df.columns = [
    "TotalOrders",
    "TotalSpent",
    "AverageSpent",
    "ItemsBought",
    "CanceledInvoices",
    "MostFrequentDay",
    "MostFrequentHour",
    "ProductDiversity",
    "FirstPurchase",
    "LastPurchase",
    "Country",
]
grouped_df.head()

,TotalOrders,TotalSpent,AverageSpent,ItemsBought,CanceledInvoices,MostFrequentDay,MostFrequentHour,ProductDiversity,FirstPurchase,LastPurchase,Country
CustomerID,,,,,,,,,,,
12346,2,0.00,0.000000,0,1,Tuesday,10,1,2011-01-18 10:01:00,2011-01-18 10:17:00,United Kingdom
12347,7,4310.00,23.681319,2458,0,Tuesday,14,103,2010-12-07 14:57:00,2011-12-07 15:52:00,Iceland
12348,4,1797.24,57.975484,2341,0,Thursday,19,22,2010-12-16 19:09:00,2011-09-25 13:13:00,Finland
12349,1,1757.55,24.076027,631,0,Monday,9,73,2011-11-21 09:51:00,2011-11-21 09:51:00,Italy
12350,1,334.40,19.670588,197,0,Wednesday,16,17,2011-02-02 16:01:00,2011-02-02 16:01:00,Norway


**Note That:** The `TotalOrders` column is the total number of invoices with the cancled invoices included, we want to remove the cancled invoices to make it just the number of orders

In [89]:
# Chnage the totaloder by romving the cancled invoices
grouped_df["TotalOrders"] = grouped_df["TotalOrders"] - grouped_df["CanceledInvoices"]
grouped_df.head()

,TotalOrders,TotalSpent,AverageSpent,ItemsBought,CanceledInvoices,MostFrequentDay,MostFrequentHour,ProductDiversity,FirstPurchase,LastPurchase,Country
CustomerID,,,,,,,,,,,
12346,1,0.00,0.000000,0,1,Tuesday,10,1,2011-01-18 10:01:00,2011-01-18 10:17:00,United Kingdom
12347,7,4310.00,23.681319,2458,0,Tuesday,14,103,2010-12-07 14:57:00,2011-12-07 15:52:00,Iceland
12348,4,1797.24,57.975484,2341,0,Thursday,19,22,2010-12-16 19:09:00,2011-09-25 13:13:00,Finland
12349,1,1757.55,24.076027,631,0,Monday,9,73,2011-11-21 09:51:00,2011-11-21 09:51:00,Italy
12350,1,334.40,19.670588,197,0,Wednesday,16,17,2011-02-02 16:01:00,2011-02-02 16:01:00,Norway


In [90]:
# Add the life time column to the data (the difference between the first and last purchase)
grouped_df["LifeTime"] = (
    grouped_df["LastPurchase"] - grouped_df["FirstPurchase"]
).dt.days
grouped_df.head()

,TotalOrders,TotalSpent,AverageSpent,ItemsBought,CanceledInvoices,MostFrequentDay,MostFrequentHour,ProductDiversity,FirstPurchase,LastPurchase,Country,LifeTime
CustomerID,,,,,,,,,,,,
12346,1,0.00,0.000000,0,1,Tuesday,10,1,2011-01-18 10:01:00,2011-01-18 10:17:00,United Kingdom,0
12347,7,4310.00,23.681319,2458,0,Tuesday,14,103,2010-12-07 14:57:00,2011-12-07 15:52:00,Iceland,365
12348,4,1797.24,57.975484,2341,0,Thursday,19,22,2010-12-16 19:09:00,2011-09-25 13:13:00,Finland,282
12349,1,1757.55,24.076027,631,0,Monday,9,73,2011-11-21 09:51:00,2011-11-21 09:51:00,Italy,0
12350,1,334.40,19.670588,197,0,Wednesday,16,17,2011-02-02 16:01:00,2011-02-02 16:01:00,Norway,0


In [91]:
# Add recency column to the data (the difference between the last purchase and the last day in the data)
grouped_df["Recency"] = (
    grouped_df["LastPurchase"].max() - grouped_df["LastPurchase"]
).dt.days
grouped_df.head()

,TotalOrders,TotalSpent,AverageSpent,ItemsBought,CanceledInvoices,MostFrequentDay,MostFrequentHour,ProductDiversity,FirstPurchase,LastPurchase,Country,LifeTime,Recency
CustomerID,,,,,,,,,,,,,
12346,1,0.00,0.000000,0,1,Tuesday,10,1,2011-01-18 10:01:00,2011-01-18 10:17:00,United Kingdom,0,325
12347,7,4310.00,23.681319,2458,0,Tuesday,14,103,2010-12-07 14:57:00,2011-12-07 15:52:00,Iceland,365,1
12348,4,1797.24,57.975484,2341,0,Thursday,19,22,2010-12-16 19:09:00,2011-09-25 13:13:00,Finland,282,74
12349,1,1757.55,24.076027,631,0,Monday,9,73,2011-11-21 09:51:00,2011-11-21 09:51:00,Italy,0,18
12350,1,334.40,19.670588,197,0,Wednesday,16,17,2011-02-02 16:01:00,2011-02-02 16:01:00,Norway,0,309


In [92]:
grouped_df.shape

(4372, 13)

In [93]:
grouped_df.to_csv("customers_data.csv")